# Federated Never Ending Learning

## Alignment Layer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

2022-11-12 21:29:05.474888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 21:29:06.257447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64/openmpi/lib
2022-11-12 21:29:06.257520: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-12 21:29:06.350479: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 21:29:08.819562:

In [4]:
df = pd.read_csv('datasets/CICIDS_test.csv', skipinitialspace=True)

In [5]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
# from keras.utils import to_categorical

# # removing any class with less than 'thresh' samples
# thresh = 1000
# counts = df[['Label']].value_counts().to_dict()
# keep = [x[0] for x in counts if counts[x] >= thresh]
# df = df.loc[df['Label'].isin(keep)]


X = df.loc[:, df.columns != 'Label']
X = Normalizer().fit_transform(X)
X = MinMaxScaler().fit_transform(X)
Y = df[['Label']].to_numpy()

In [6]:
print(X.shape)
print(Y.shape)
print(len(np.unique(Y)))
pd.value_counts(Y.ravel())

(566149, 78)
(566149, 1)
15


0     454620
4      46215
10     31786
2      25606
3       2059
7       1588
11      1179
6       1159
5       1100
1        393
12       301
14       130
9          7
13         4
8          2
dtype: int64

## Non-Federated

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, random_state=1)

print(X_train.shape)
print(Y_train.shape)
print(np.unique(Y_train))
print(pd.value_counts(Y_train.ravel()))

print(X_test.shape)
print(Y_test.shape)
print(np.unique(Y_test))
print(pd.value_counts(Y_test.ravel()))

(396304, 78)
(396304, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
0     318395
4      32391
10     22137
2      17891
3       1417
7       1123
11       827
6        786
5        780
1        257
12       197
14        92
9          7
13         2
8          2
dtype: int64
(169845, 78)
(169845, 1)
[ 0  1  2  3  4  5  6  7 10 11 12 13 14]
0     136225
4      13824
10      9649
2       7715
3        642
7        465
6        373
11       352
5        320
1        136
12       104
14        38
13         2
dtype: int64


### Binary Classification into Benign/Malicious

In [42]:
class BinaryClassification:
    def __init__(self, input_dims, layers):
        self.model = Sequential(name="server_model")
        self.model.add(Dense(
            layers[0],
            input_shape=(input_dims,),
            activation='relu'
        ))
        for l in layers[1:]:
            self.model.add(Dense(l, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy', ]
        )
        self.model.summary()
    
    def fit(self, X, Y, epochs, batch_size):
        Y_bin = np.asarray([
            0 if y == 0 else 1 for y in Y.ravel()
        ]).reshape(-1, 1)
        # print(Y_bin.shape)
        # print(pd.value_counts(Y_bin.ravel()))
        self.model.fit(X, Y_bin, epochs=epochs, batch_size=batch_size)

    def predict(self, X, Y):
        Y_bin = np.asarray([
            0 if y == 0 else 1 for y in Y.ravel()
        ]).reshape(-1, 1)
        print(pd.value_counts(Y_bin.ravel()))
        
        preds = self.model.predict(X)
        preds = np.where(preds > 0.5, 1, 0)

        print(Y_bin.shape)
        print(preds.shape)
        print(classification_report(Y_bin, preds))
        print(confusion_matrix(Y_bin, preds))

In [25]:
model_binary = BinaryClassification(
    input_dims=78,
    layers=[20, 20, 20]
)
model_binary.fit(X_train, Y_train, epochs=1, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 20)                1580      
                                                                 
 dense_19 (Dense)            (None, 20)                420       
                                                                 
 dense_20 (Dense)            (None, 20)                420       
                                                                 
 dense_21 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
(396304, 1)
0    318395
1     77909
dtype: int64
12385/12385 [==============================] - 40s 3ms/step - loss: 0.1030 - accuracy: 0.9614
(169845, 1)
0    136225
1     33620
dtype

In [26]:
model_binary.predict(X_test, Y_test)

(169845, 1)
0    136225
1     33620
dtype: int64
5308/5308 [==============================] - 13s 2ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    136225
           1       0.97      0.95      0.96     33620

    accuracy                           0.98    169845
   macro avg       0.98      0.97      0.97    169845
weighted avg       0.98      0.98      0.98    169845

[[135138   1087]
 [  1767  31853]]


### SoftMax A

In [50]:
class MulticlassClassification:
    def __init__(self, input_dims, layers, num_classes):
        self.model = Sequential(name="server_model")
        self.model.add(Dense(
            layers[0],
            input_shape=(input_dims,),
            activation='relu'
        ))
        for l in layers[1:]:
            self.model.add(Dense(l, activation='relu'))
        self.model.add(Dense(num_classes, activation='softmax'))
        self.model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        self.model.summary()
    
    def fit(self, X, Y, epochs, batch_size):
        print(pd.value_counts(Y.ravel()))
        self.model.fit(X, Y.ravel(), epochs=epochs, batch_size=batch_size)

    def predict(self, X, Y):
        print(pd.value_counts(Y.ravel()))
        
        preds = self.model.predict(X)
        preds = [np.argmax(p) for p in preds]
        
        print(classification_report(Y, preds))
        print(confusion_matrix(Y, preds))

In [30]:
model_sf = MulticlassClassification(
    input_dims=78,
    layers=[20, 20, 20],
    num_classes=15
)
model_sf.fit(X_train, Y_train, epochs=1, batch_size=32)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 20)                1580      
                                                                 
 dense_27 (Dense)            (None, 20)                420       
                                                                 
 dense_28 (Dense)            (None, 20)                420       
                                                                 
 dense_29 (Dense)            (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
0     318395
4      32391
10     22137
2      17891
3       1417
7       1123
11       827
6        786
5        780
1        257
12       197
14        92
9          7
13         2
8 

In [31]:
model_sf.predict(X_test, Y_test)

0     136225
4      13824
10      9649
2       7715
3        642
7        465
6        373
11       352
5        320
1        136
12       104
14        38
13         2
dtype: int64
5308/5308 [==============================] - 9s 2ms/step


/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99    136225
           1       0.00      0.00      0.00       136
           2       0.92      0.94      0.93      7715
           3       0.93      0.63      0.75       642
           4       0.91      0.97      0.94     13824
           5       0.76      0.52      0.62       320
           6       0.94      0.29      0.44       373
           7       0.92      0.80      0.86       465
          10       0.98      0.94      0.96      9649
          11       0.82      0.96      0.88       352
          12       0.00      0.00      0.00       104
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00        38

    accuracy                           0.98    169845
   macro avg       0.63      0.54      0.57    169845
weighted avg       0.97      0.98      0.97    169845

[[134528      0    258     20   1094     42      0     12    207     64
       

## Anomaly Detection Using Isolation Forest

## Federated Learning

In [34]:
X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, train_size=0.5, random_state=1)

In [35]:
print(X_1.shape)
print(Y_1.shape)
print(np.unique(Y_1))
print(pd.value_counts(Y_1.ravel()))
print(X_2.shape)
print(Y_2.shape)
print(np.unique(Y_2))
print(pd.value_counts(Y_2.ravel()))

(283074, 78)
(283074, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
0     227375
4      23136
10     15862
2      12759
3       1016
7        802
11       595
5        575
6        560
1        178
12       138
14        68
9          7
8          2
13         1
dtype: int64
(283075, 78)
(283075, 1)
[ 0  1  2  3  4  5  6  7 10 11 12 13 14]
0     227245
4      23079
10     15924
2      12847
3       1043
7        786
6        599
11       584
5        525
1        215
12       163
14        62
13         3
dtype: int64


In [36]:
X_1_train, X_1_test, Y_1_train, Y_1_test = train_test_split(X_1, Y_1, train_size=0.7, random_state=1)
X_2_train, X_2_test, Y_2_train, Y_2_test = train_test_split(X_2, Y_2, train_size=0.7, random_state=1)

In [37]:
print(X_1_train.shape)
print(Y_1_train.shape)
print(np.unique(Y_1_train))
# print(np.isnan(X_1_train).any())
# print(np.isnan(Y_1_train).any())

print(X_1_test.shape)
print(Y_1_test.shape)
print(np.unique(Y_1_test))

print(X_2_train.shape)
print(Y_2_train.shape)
print(np.unique(Y_2_train))

print(X_2_test.shape)
print(Y_2_test.shape)
print(np.unique(Y_2_test))

(198151, 78)
(198151, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
(84923, 78)
(84923, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 14]
(198152, 78)
(198152, 1)
[ 0  1  2  3  4  5  6  7 10 11 12 13 14]
(84923, 78)
(84923, 1)
[ 0  1  2  3  4  5  6  7 10 11 12 14]


### Binary Classification

In [43]:
model_1 = BinaryClassification(78, [20, 20, 20])
model_1.fit(X_1_train, Y_1_train, epochs=1, batch_size=32)
model_1.predict(X_1_test, Y_1_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 20)                1580      
                                                                 
 dense_47 (Dense)            (None, 20)                420       
                                                                 
 dense_48 (Dense)            (None, 20)                420       
                                                                 
 dense_49 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
6193/6193 [==============================] - 19s 3ms/step - loss: 0.1341 - accuracy: 0.9461
0    68256
1    16667
dtype: int64
2654/2654 [==============================] - 6s 2ms/step


In [46]:
model_2 = BinaryClassification(78, [20, 20, 20])
model_2.fit(X_2_train, Y_2_train, epochs=1, batch_size=32)
model_2.predict(X_2_test, Y_2_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 20)                1580      
                                                                 
 dense_59 (Dense)            (None, 20)                420       
                                                                 
 dense_60 (Dense)            (None, 20)                420       
                                                                 
 dense_61 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
6193/6193 [==============================] - 17s 3ms/step - loss: 0.1423 - accuracy: 0.9438
0    67980
1    16943
dtype: int64
2654/2654 [==============================] - 6s 2ms/step


In [49]:
model_3 = BinaryClassification(78, [20, 20, 20])
model_3.model.set_weights(np.mean(
        np.array([
            model_1.model.get_weights(),
            model_2.model.get_weights()
        ]),
        axis=0
    )
)
model_3.predict(X_1_test, Y_1_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 20)                1580      
                                                                 
 dense_71 (Dense)            (None, 20)                420       
                                                                 
 dense_72 (Dense)            (None, 20)                420       
                                                                 
 dense_73 (Dense)            (None, 1)                 21        
                                                                 
Total params: 2,441
Trainable params: 2,441
Non-trainable params: 0
_________________________________________________________________
0    68256
1    16667
dtype: int64


/tmp/ipykernel_125746/1243486425.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([


2654/2654 [==============================] - 7s 3ms/step
(84923, 1)
(84923, 1)
              precision    recall  f1-score   support

           0       0.91      0.93      0.92     68256
           1       0.68      0.61      0.64     16667

    accuracy                           0.87     84923
   macro avg       0.79      0.77      0.78     84923
weighted avg       0.86      0.87      0.86     84923

[[63449  4807]
 [ 6478 10189]]


### Multiclass Classification

In [51]:
model_sf_1 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_1.fit(X_1_train, Y_1_train, epochs=1, batch_size=32)
model_sf_1.predict(X_1_test, Y_1_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 20)                1580      
                                                                 
 dense_75 (Dense)            (None, 20)                420       
                                                                 
 dense_76 (Dense)            (None, 20)                420       
                                                                 
 dense_77 (Dense)            (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
0     159119
4      16268
10     11038
2       8969
3        708
7        550
11       436
6        393
5        388
1        129
12        97
14        49
9          5
8          1
13

/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
model_sf_2 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_2.fit(X_2_train, Y_2_train, epochs=1, batch_size=32)
model_sf_2.predict(X_2_test, Y_2_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 20)                1580      
                                                                 
 dense_79 (Dense)            (None, 20)                420       
                                                                 
 dense_80 (Dense)            (None, 20)                420       
                                                                 
 dense_81 (Dense)            (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
0     159265
4      16106
10     11074
2       8919
3        753
7        534
11       415
6        406
5        368
1        152
12       117
14        40
13         3
dtype: int64
61

/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
model_sf_3 = MulticlassClassification(
    78,
    [20, 20, 20],
    num_classes=15
)
model_sf_3.model.set_weights(np.mean(
        np.array([
            model_sf_1.model.get_weights(),
            model_sf_2.model.get_weights()
        ]),
        axis=0
    )
)
model_sf_3.predict(X_1_test, Y_1_test)

Model: "server_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 20)                1580      
                                                                 
 dense_83 (Dense)            (None, 20)                420       
                                                                 
 dense_84 (Dense)            (None, 20)                420       
                                                                 
 dense_85 (Dense)            (None, 15)                315       
                                                                 
Total params: 2,735
Trainable params: 2,735
Non-trainable params: 0
_________________________________________________________________
0     68256
4      6868
10     4824
2      3790
3       308
7       252
5       187
6       167
11      159
1        49
12       41
14       19
9         2
8         1
dtype: int64


/tmp/ipykernel_125746/1574118328.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([


2654/2654 [==============================] - 5s 2ms/step
              precision    recall  f1-score   support

           0       0.83      0.91      0.87     68256
           1       0.00      0.00      0.00        49
           2       0.00      0.00      0.00      3790
           3       0.00      0.00      0.00       308
           4       0.00      0.00      0.00      6868
           5       0.00      0.00      0.00       187
           6       0.00      0.03      0.00       167
           7       0.00      0.00      0.00       252
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2
          10       0.48      0.61      0.54      4824
          11       0.00      0.00      0.00       159
          12       0.00      0.00      0.00        41
          14       0.00      0.00      0.00        19

    accuracy                           0.77     84923
   macro avg       0.09      0.11      0.10     84923
weighted avg       0.69

/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
